In [1]:
import os
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Coron3Pipeline
import jwst.associations
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from glob import glob
import astropy.io.fits as fits
import numpy as np
import time
import pandas as pd
import json
import pickle
from json import JSONEncoder
from collections import defaultdict
from CoronPipeline import MyCoron3Pipeline

In [2]:
os.environ["CRDS_PATH"] = '/home/sarperyn/crds_cache/jwst_ops'
os.environ["CRDS_SERVER_URL"] = 'https://jwst-crds.stsci.edu'

In [3]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

# figure sizes and resolutions
rcParams['figure.figsize'] = [20,15]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 80

In [4]:
import jwst
print(jwst.__version__)

1.8.3.dev39+g731bc17


In [25]:
PROPOSAL_ID = '1536'
INSTRUME = 'NIRCAM'
#DETECTOR = 'nrca2'

# Pipeline things



<div class="alert alert-block alert-info">
Now we have the fits files. We will create our coronagrahic images using the JWST pipeline. Our data has the suffix <b>.rateints</b> just like we needed. 
While processing coronagraphy data, the common procedure is using the following pipeline:
    <ul> 
        <li> calwebb_detector1 $\rightarrow$ <strong>from jwst.pipeline import Detector1Pipeline</strong>
        <li> calwebb_image2  $\rightarrow$ <strong>from jwst.pipeline import Image2Pipeline</strong>
        <li> calwebb_coron3  $\rightarrow$ <strong>from jwst.pipeline import Coron3Pipeline</strong>
    </ul>
</div>

We have the outputs of the first stage. They all have the `rateints.fits` suffix. By passing them through the second pipeline we are going to end up with `calints.fits` files which is a last step before getting the coron products.

In [26]:
# Path containing the rateints data
directory = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/{INSTRUME}/{PROPOSAL_ID}/mastDownload/JWST/'

In [7]:
directory

'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1536/mastDownload/JWST/'

### Stage2

In [8]:
def get_exptypes(fits_):
    
    for file in range(len(fits_)):
        
        f = fits.open(fits_[file])
        exp_type = f[0].header['EXP_TYPE']
        print(exp_type)

In [9]:
def runimg2(filename, output_dir):
    
    img2 = Image2Pipeline()
    img2.output_dir = output_dir
    img2.save_results = True
    img2(filename)

In [10]:
def delete_files(files):
    for file in files:

        if os.path.isfile(file):
            os.remove(file)
            print("File has been deleted")
        else:
            print("File does not exist")

In [33]:
rateints_files = glob(os.path.join(directory, '**/*nrca2_rateints.fits'))

In [34]:
len(rateints_files)

25

In [35]:
rateints_files[0][:114]

'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1536/mastDownload/JWST/jw01536064001_03105_00001_nrca2/'

In [36]:
batch_size = 4

In [37]:
for i in range(0,len(rateints_files),batch_size):
    
    for f in rateints_files[i:i+batch_size]:
        
        output_dir = f[:114]
        runimg2(f,output_dir)
    time.sleep(1)

2023-01-21 08:44:52,890 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.
2023-01-21 08:44:52,893 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2023-01-21 08:44:52,895 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2023-01-21 08:44:52,896 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2023-01-21 08:44:52,898 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.
2023-01-21 08:44:52,899 - stpipe.Image2Pipeline.resample - INFO - ResampleStep instance created.
2023-01-21 08:44:53,224 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline running with args ('/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1536/mastDownload/JWST/jw01536064001_03105_00001_nrca2/jw01536064001_03105_00001_nrca2_rateints.fits',).
2023-01-21 08:44:53,230 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'outpu

In [73]:
calints_data = glob(os.path.join(directory, '**/*nrcalong_calints.fits'))

In [74]:
calints1 = fits.open(calints_data[0])
calints1.info()

Filename: /home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1536/mastDownload/JWST/jw01536066001_03107_00005_nrcalong/jw01536066001_03107_00005_nrcalong_calints.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     356   ()      
  1  SCI           1 ImageHDU        97   (320, 320, 1)   float32   
  2  ERR           1 ImageHDU        11   (320, 320, 1)   float32   
  3  DQ            1 ImageHDU        12   (320, 320, 1)   int32 (rescales to uint32)   
  4  AREA          1 ImageHDU         9   (320, 320)   float32   
  5  INT_TIMES     1 BinTableHDU     24   1R x 7C   [J, D, D, D, D, D, D]   
  6  VAR_POISSON    1 ImageHDU        10   (320, 320, 1)   float32   
  7  VAR_RNOISE    1 ImageHDU        10   (320, 320, 1)   float32   
  8  VAR_FLAT      1 ImageHDU        10   (320, 320, 1)   float32   
  9  ASDF          1 BinTableHDU     11   1R x 1C   [26102B]   


### Stage 3

The stage 3 is kinda different than the other steps. To run this step we need an association (ASN) file. Thus, we need some functions to create ASN files.

In [75]:
len(calints_data)

143

In [76]:
from jwst.associations.mkpool import mkpool
from jwst.associations.lib.rules_level3 import Asn_Lv3Coron
from jwst.associations import AssociationPool, AssociationRegistry
from jwst.associations.mkpool import from_cmdline, mkpool
from jwst.associations import generate
from astropy.table import Table

In [77]:
pool = mkpool(calints_data)

In [78]:
pool_df = pool.to_pandas()
pool_df.to_csv(f'calints_{INSTRUME}_{PROPOSAL_ID}_pool.csv',index=False)

In [79]:
def t_path(partial_path):
    __file__ = '/home/sarperyn/sarperyurtseven/ProjectFiles/notebooks/asn_coron_rule.py'
    test_dir = os.path.dirname(__file__)
    return os.path.join(test_dir, partial_path)

In [80]:
registry = AssociationRegistry([t_path('asn_coron_rule.py')],
                              include_default=False)

In [81]:
asns = generate(pool,registry)

202301210906:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1536/mastDownload/JWST/jw01536066001_03107_00005_nrcalong/jw01536066001_03107_00005_nrcalong_calints.fits" is not a Level 1b name. Cannot transform to Level 2b.
202301210906:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1536/mastDownload/JWST/jw01536066001_0310l_00001_nrcalong/jw01536066001_0310l_00001_nrcalong_calints.fits" is not a Level 1b name. Cannot transform to Level 2b.
202301210906:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1536/mastDownload/JWST/jw01536066001_03105_00004_nrcalong/jw01536066001_03105_00004_nrcalong_calints.fits" is not a Level 1b name. Cannot transform to Level 2b.
202301210906:WARNING:jwst.associations.lib.rules_level3_base.

In [82]:
len(asns)

0

In [83]:
asns

[]

In [27]:
class FileEncoder(JSONEncoder):
        def default(self, o):
            return o.__dict__

In [28]:
def get_asns(asns):
    
    asn_list = []
    for idx,asn in enumerate(asns):
        asn_dict = {}
        
        for i,j in zip(asn.keys(),asn.values()):
            asn_dict[i] = j
            
        asn_list.append(asn_dict)
        
        
        # I couldn't figure out how to serialize the abc.AsnCoron_Lv3 object and save it as a json file
        # For a temporary solution we will have a list that contains asn files as dictionaries
        # In this way, we can easily run the pipeline
        '''
        FileEncoder().encode(asn_dict)
        with open(f'asnfile_{idx}.json','w') as output:
            encodedJSON = json.dump(asn_dict,output,indent=4,cls=FileEncoder)
        '''
        
    return asn_list
            

In [29]:
def runcoron(filename, output_dir):

    coron = MyCoron3Pipeline()
    coron.output_dir = output_dir
    coron.save_results = True
    coron.process(filename)

In [30]:
def get_stage3_products(asns,directory):
    
    for t in range(len(asns)):

        asn_dict = {}
        for i,j in zip(asns[t].keys(),asns[t].values()):
            asn_dict[i] = j
            
        runcoron(asn_dict,directory)
        
        

In [31]:
print(len(asns[0]['products'][0]['members']))
print(len(asns[1]['products'][0]['members']))
#print(len(asns[2]['products'][0]['members']))

17
17


In [ ]:
get_stage3_products(asns,directory)

2023-01-21 05:44:25,968 - stpipe.MyCoron3Pipeline - INFO - MyCoron3Pipeline instance created.
2023-01-21 05:44:25,970 - stpipe.MyCoron3Pipeline.stack_refs - INFO - StackRefsStep instance created.
2023-01-21 05:44:25,973 - stpipe.MyCoron3Pipeline.align_refs - INFO - AlignRefsStep instance created.
2023-01-21 05:44:25,975 - stpipe.MyCoron3Pipeline.klip - INFO - KlipStep instance created.
2023-01-21 05:44:25,978 - stpipe.MyCoron3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2023-01-21 05:44:25,982 - stpipe.MyCoron3Pipeline.resample - INFO - ResampleStep instance created.
2023-01-21 05:44:25,983 - stpipe.MyCoron3Pipeline - INFO - Starting calwebb_coron3 ...


Output: jw01441-a3001_t004_nircam_f250m-maskbar-sub320alwb
Prod: {'name': 'jw01441-a3001_t004_nircam_f250m-maskbar-sub320alwb', 'members': [{'expname': '/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1441/mastDownload/JWST/jw01441008001_03108_00002_nrcalong/jw01441008001_03108_00002_nrcalong_calints.fits', 'exptype': 'psf', 'exposerr': None, 'asn_candidate': "[('o008', 'observation')]"}, {'expname': '/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1441/mastDownload/JWST/jw01441007001_03108_00003_nrcalong/jw01441007001_03108_00003_nrcalong_calints.fits', 'exptype': 'psf', 'exposerr': None, 'asn_candidate': "[('o007', 'observation')]"}, {'expname': '/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/1441/mastDownload/JWST/jw01441006001_03108_00001_nrcalong/jw01441006001_03108_00001_nrcalong_calints.fits', 'exptype': 'science', 'exposerr': None, 'asn_candidate': "[('o006', 'observation')]"}, {'expname': '/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NI

2023-01-21 05:44:36,859 - stpipe.MyCoron3Pipeline - INFO - Prefetching reference files for dataset: 'jw01441008001_03108_00002_nrcalong_calints.fits' reftypes = ['drizpars', 'psfmask']
2023-01-21 05:44:37,318 - CRDS - INFO -  Fetching  /home/sarperyn/crds_cache/jwst_ops/mappings/jwst/jwst_nirspec_readnoise_0025.rmap    2.6 K bytes  (1 / 10 files) (0 / 80.5 K bytes)
2023-01-21 05:44:37,422 - CRDS - INFO -  Fetching  /home/sarperyn/crds_cache/jwst_ops/mappings/jwst/jwst_nirspec_fflat_0017.rmap    7.2 K bytes  (2 / 10 files) (2.6 K / 80.5 K bytes)
2023-01-21 05:44:37,630 - CRDS - INFO -  Fetching  /home/sarperyn/crds_cache/jwst_ops/mappings/jwst/jwst_nirspec_0308.imap    5.1 K bytes  (3 / 10 files) (9.8 K / 80.5 K bytes)
2023-01-21 05:44:37,714 - CRDS - INFO -  Fetching  /home/sarperyn/crds_cache/jwst_ops/mappings/jwst/jwst_nircam_superbias_0014.rmap   11.9 K bytes  (4 / 10 files) (14.9 K / 80.5 K bytes)
2023-01-21 05:44:37,812 - CRDS - INFO -  Fetching  /home/sarperyn/crds_cache/jwst_ops